In [1]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd

In [2]:
print(ee.__version__)
print(geemap.__version__)
ee.Initialize()

0.1.221
0.6.13


# 1) Load Venus shapes

In [3]:
manaus_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)
info = manaus.geometry()
manaus = ee.Geometry(info)

atto_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_atto_footprint/venus_atto_footprint.shp'
atto = geemap.shp_to_ee(atto_shp)
info = atto.geometry()
atto = ee.Geometry(info)

corumba_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_corumba_footprint/venus_corumba_footprint.shp'
corumba = geemap.shp_to_ee(corumba_shp)
info = corumba.geometry()
corumba = ee.Geometry(info)

mato_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_mato_footprint/venus_mato_footprint.shp'
mato = geemap.shp_to_ee(mato_shp)
info = mato.geometry()
mato = ee.Geometry(info)

saop_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_saop_footprint/venus_saop_footprint.shp'
saop = geemap.shp_to_ee(saop_shp)
info = saop.geometry()
saop = ee.Geometry(info)

parana_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_parana_footprint/venus_parana_footprint.shp'
parana = geemap.shp_to_ee(parana_shp)
info = parana.geometry()
parana = ee.Geometry(info)

comodo_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_comodo_footprint/venus_comodo_footprint.shp'
comodo = geemap.shp_to_ee(comodo_shp)
info = comodo.geometry()
comodo = ee.Geometry(info)

k34_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_k34_footprint/venus_k34_footprint.shp'
k34 = geemap.shp_to_ee(k34_shp)
info = k34.geometry()
k34 = ee.Geometry(info)

jam_shp = '/home/jbk/projects/climate/tropics/venus/footprints/venus_jam_footprint/venus_jam_footprint.shp'
jam = geemap.shp_to_ee(jam_shp)
info = jam.geometry()
jam = ee.Geometry(info)

# 2) Load Rondonia

In [3]:
# Import grid
sitename = 'rondonia'
grid_shp = '/data/6ru/shapefiles/rondonia_test.shp'
grid_fc = geemap.shp_to_ee(grid_shp)

# Extract coordinates from feature collection
grid_fc_dict = grid_fc.getInfo()
feats = grid_fc_dict['features']
coord_list = []
for d in feats:
    coords = d['geometry']['coordinates']
    coord_list.append(coords)

# Create polygons from coordinates
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

# Select only the first 100 grid squares
#idx = list(range(0,100))
#polys = [ polys[i] for i in idx]

# Select one of first 100 grid squares
geom = polys[0]
fc = ee.FeatureCollection(geom)
print(geom.getInfo())

{'type': 'Polygon', 'coordinates': [[[-64.72361622081152, -9.881468125314736], [-64.22954281454578, -9.881468125314736], [-64.22954281454578, -9.387394719048999], [-64.72361622081152, -9.387394719048999], [-64.72361622081152, -9.881468125314736]]]}


# 3) Load boundaries #

In [61]:
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
       .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
       .filterMetadata('ADM1_NAME', 'equals', 'Amazonas')
      )

# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools
def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

# Make test grid (half degree squares)
grid_55km = make_grid(br, 40000)

# Access coordinates of grid squares
grid_dict = grid_55km.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)
    
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)
    
# Make grid smaller if it's huge
idx = [0]
polys = [ polys[i] for i in idx]

# Make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys)
grid_fc = grid
fc = grid

In [66]:
col = (ee.ImageCollection('COPERNICUS/S2')
       .filterDate('2017-01-01', '2020-12-31')
       .sort('CLOUDY_PIXEL_PERCENTAGE', False))

col = col.map(lambda image: image.clip(grid_fc))

Map = geemap.Map()
Map.add_basemap("SATELLITE")
Map.addLayer(grid_fc, {}, 'grid')
Map.addLayer(col, {'bands': ['B4', 'B3', 'B2'], 'min':0, 'max':3000}, 'col')
Map.center_object(grid_fc, zoom=8)
Map

Map(center=[-2.3356235491202306, -56.59386289953019], controls=(WidgetControl(options=['position'], widget=HBo…

In [67]:
SDATE = '2017-01-01'
EDATE = '2020-12-31'

# Daily mean 2m air temperature
era5_2mt = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                   .select('mean_2m_air_temperature')
                   .filterBounds(fc)
                   .filter(ee.Filter.date(SDATE, EDATE)))
#print(era5_2mt))

# Daily total precipitation sums
era5_tp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('total_precipitation')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE)))

era5_2mt = era5_2mt.map(lambda img: img.clip(fc))
era5_tp = era5_tp.map(lambda img: img.clip(fc))

# Visualization palette for total precipitation
visTp = {
  'min': 0.00,
  'max': 0.03,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis2mt = {
  'min': 297,
  'max': 303,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

Map.addLayer(era5_2mt, vis2mt, 'air temp')
Map.addLayer(era5_tp, visTp, 'precip')

In [68]:
years = ee.List.sequence(2017, 2020)
step = ee.List.sequence(1, 365, 15)

clipped_cols = [era5_2mt, era5_tp]
all_cols = []
for a_col in clipped_cols:
    def byYear(y):
        y = ee.Number(y)
        def byStep(d):
            d = ee.Number(d)
            return (a_col
                    .filter(ee.Filter.calendarRange(y, y, 'year')) #yearly step
                    .filter(ee.Filter.calendarRange(d, d.add(14), 'day_of_year')) #15-day step
                    .mean()
                    .set('step', [d, y])) #Add properties
                
        return step.map(byStep)

    col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
    all_cols.append(col)
    
column = all_cols[0].filter(ee.Filter.eq('system:index', '0'))
print(column.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'mean_2m_air_temperature', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'step': [1, 2017], 'system:index': '0'}}]}


In [69]:
# Make a list of file names
tiles = []
sitename = 'am_all'
for num in range(len(all_cols)):
    index = str(sitename + '_{}'.format(num))
    tiles.append(index)

print(tiles)

['am_all_0', 'am_all_1']


In [13]:
# Export monthly images from a collection
import os, time

polys = [polys[0], polys[0]]
tic1 = time.time()
for a_col, a_tile, poly in zip(all_cols, tiles, polys):
    ilist = a_col.toList(a_col.size())
    for i in range(0,25):
        if len(ee.Image(ilist.get(i)).bandNames().getInfo()) <= 0:
            print("ERROR; No bands found in image index %d... will skip export."%(i))
        else:
            filename = "/Users/morgansteckler/Desktop/{}/{}.tif".format(a_tile,i)
            temp_dir = "/Users/morgansteckler/Desktop/{}/".format(a_tile)
            if not os.path.exists(temp_dir):
                os.mkdir(temp_dir)
            if os.path.exists(filename):
                print("Image {} already exists. Skipping...".format(i))
                next
            else:
                tic = time.time()
                print("Exporting Image %d"%(i))
                geemap.ee_export_image(ee.Image(ilist.get(i)), 
                                       filename=filename, 
                                       scale=30, 
                                       region=poly, 
                                       file_per_band=False)
                toc = time.time()
                hours, rem = divmod(toc-tic, 3600)
                mins, secs = divmod(rem, 60)
                print("Time elapsed: {:0>2}:{:0>2}:{:05.2f}"
                      .format(int(hours),int(mins),secs))
toc1 = time.time()
hrs1, rem1 = divmod(toc1-tic1, 3600)
mins1, secs1 = divmod(rem1,  60)
print("Total time elapsed: {:0>2}:{:0>2}:{:05.2f}"
      .format(int(hrs1),int(mins1),secs1))

Image 0 already exists. Skipping...
Image 1 already exists. Skipping...
Image 2 already exists. Skipping...
Image 3 already exists. Skipping...
Image 4 already exists. Skipping...
Image 5 already exists. Skipping...
Image 6 already exists. Skipping...
Image 7 already exists. Skipping...
Image 8 already exists. Skipping...
Image 9 already exists. Skipping...
Image 10 already exists. Skipping...
Image 11 already exists. Skipping...
Image 12 already exists. Skipping...
Image 13 already exists. Skipping...
Image 14 already exists. Skipping...
Image 15 already exists. Skipping...
Image 16 already exists. Skipping...
Image 17 already exists. Skipping...
Image 18 already exists. Skipping...
Image 19 already exists. Skipping...
Image 20 already exists. Skipping...
Image 21 already exists. Skipping...
Image 22 already exists. Skipping...
Image 23 already exists. Skipping...
Image 24 already exists. Skipping...
Exporting Image 0
Generating URL ...
Please wait ...
Data downloaded to /Users/morgan

Data downloaded to /Users/morgansteckler/Desktop/ro10_1_2018/22.tif
Time elapsed: 00:00:07.13
Exporting Image 23
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/ro10_1_2018/23.tif
Time elapsed: 00:00:06.72
Exporting Image 24
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/ro10_1_2018/24.tif
Time elapsed: 00:00:02.81
Total time elapsed: 00:02:48.61


# End Roraima #

## Define some processing routines 

In [70]:
# Mean

RES=30

def region_mean_Image(img):
    # mean over all pixels in the region for the image
    return img.set('mean', img.reduceRegion(ee.Reducer.mean(), geometry=geom, scale=RES)) 


def region_mean_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_mean_Image)
    stats_list = stats.aggregate_array('mean')
    return np.array(stats_list.getInfo())

# Median
def region_median_Image(img):
    # mean over all pixels in the region for the image
    return img.set('median', img.reduceRegion(ee.Reducer.median(), geometry=geom, scale=RES)) 

def region_median_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_median_Image)
    stats_list = stats.aggregate_array('median')
    return np.array(stats_list.getInfo())

# Stddev
def region_stddev_Image(img):
    # stddev over all pixels in the region for the image
    return img.set('stddev', img.reduceRegion(ee.Reducer.stdDev(), geometry=geom, scale=RES)) 

def region_stddev_ImageCollection(ic):
    # stddev over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_stddev_Image)
    stats_list = stats.aggregate_array('stddev')
    return np.array(stats_list.getInfo())

# Percentiles
def region_percentiles_Image(img):
    # 5,10,25,75,90,95 percentiles over all pixels in the region for the image
    return img.set('percentiles', img.reduceRegion(ee.Reducer.percentile([5,10,25,75,90,95]), geometry=fc, scale=20)) #) mean, median, stdDev, percentiles

def region_percentiles_ImageCollection(ic):
    # percentiles over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_percentiles_Image)
    stats_list = stats.aggregate_array('percentiles')
    return np.array(stats_list.getInfo())

In [5]:
#Append ndre percentile dict values to new lists
def p_list(alist, varname):
    p10, p25, p5, p75, p90, p95 = [], [], [], [], [], []
    for value in alist:
        p5.append(value[varname+'_p5'])
        p10.append(value[varname+'_p10'])
        p25.append(value[varname+'_p25'])
        p75.append(value[varname+'_p75'])
        p90.append(value[varname+'_p90'])
        p95.append(value[varname+'_p95'])
    return p5, p10, p25, p75, p90, p95

In [71]:
# get list of dates from ImageCollection
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())

In [72]:
#Take only the ndre mean, med, stddev dict values and append to new list
def newList(alist, varname):
    list_name = []
    for value in alist:
        list_name.append(value[varname])
    return list_name

# ########################################
## ERA5
# ########################################

In [73]:
# Read the ERA5 data 
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY

SDATE = '2017-01-01'
EDATE = '2020-12-31'

# Daily mean 2m air temperature
era5_2mt = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                   .select('mean_2m_air_temperature')
                   .filterBounds(fc)
                   .filter(ee.Filter.date(SDATE, EDATE)))
#print(era5_2mt))

# Daily total precipitation sums
era5_tp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('total_precipitation')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE)))

# Daily mean 2m dewpoint temperature
era5_2d = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('dewpoint_2m_temperature')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE)))

# Daily mean sea-level pressure
era5_mslp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                    .select('mean_sea_level_pressure')
                    .filterBounds(fc)
                    .filter(ee.Filter.date(SDATE, EDATE)))
# Convert pressure levels from Pa to hPa - Example for surface pressure
def era5_sp(image):
    return image.divide(100).set('system:time_start', image.get('system:time_start'))

# Daily mean surface pressure
era5_sp = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                  .select('surface_pressure')
                  .filterBounds(fc)
                  .filter(ee.Filter.date(SDATE, EDATE))).map(era5_sp)


# Daily mean 10m u-component of wind
era5_u_wind_10m = (ee.ImageCollection('ECMWF/ERA5/DAILY')
                          .select('u_component_of_wind_10m')
                          .filterBounds(fc)
                          .filter(ee.Filter.date(SDATE, EDATE)))

In [74]:
# Visualization palette for total precipitation
visTp = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis2mt = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

# Visualization palette for u- and v-component of 10m wind
visWind = {
  'min': 0,
  'max': 30,
  'palette': [
    '#FFFFFF', '#FFFF71', '#DEFF00', '#9EFF00', '#77B038', '#007E55', '#005F51',
    '#004B51', '#013A7B', '#023AAD'
  ]
}

# Visualization palette for pressure (surface pressure, mean sea level
# pressure) - adjust min and max values for mslp to min:990 and max:1050
visPressure = {
  'min': 500,
  'max': 1150,
  'palette': [
    '#01FFFF', '#058BFF', '#0600FF', '#DF00FF', '#FF00FF', '#FF8C00', '#FF8C00'
  ]
}


In [24]:
Map = geemap.Map()
Map.center_object(grid, zoom=10)
Map

Map(center=[-4.158708756590903, -64.70498486280152], controls=(WidgetControl(options=['position'], widget=HBox…

In [25]:
era5_2mt = era5_2mt.map(lambda img: img.clip(grid_fc))
Map.addLayer(era5_2mt, vis2mt, 'Daily mean 2m air temperature')

In [18]:
#tp_percentile_list

In [75]:
# total precipitation
# get area statistics
tp_mean_list = region_mean_ImageCollection(era5_tp)
tp_median_list = region_median_ImageCollection(era5_tp)
tp_stddev_list = region_stddev_ImageCollection(era5_tp)
#tp_percentile_list = region_percentiles_ImageCollection(era5_tp)

# turn it into numpy array
tp_mean = newList(tp_mean_list, 'total_precipitation')
tp_median = newList(tp_median_list, 'total_precipitation')
tp_stddev = newList(tp_stddev_list, 'total_precipitation')
#tp_percentile = p_list(tp_percentile_list, 'total_precipitation')

In [76]:
# 2m air temperature
# get area statistics
t2m_mean_list = region_mean_ImageCollection(era5_2mt)
t2m_median_list = region_median_ImageCollection(era5_2mt)
t2m_stddev_list = region_stddev_ImageCollection(era5_2mt)
#t2m_percentile_list = region_percentiles_ImageCollection(era5_2mt)

# turn it into numpy array
t2m_mean = newList(t2m_mean_list, 'mean_2m_air_temperature')
t2m_median = newList(t2m_median_list, 'mean_2m_air_temperature')
t2m_stddev = newList(t2m_stddev_list, 'mean_2m_air_temperature')
#t2m_percentile = p_list(t2m_percentile_list, 'mean_2m_air_temperature')

In [ ]:
era5dates = ymdList(era5_tp)
x = range(len(t2m_mean))
plus_error = [x + y for x,y in zip(t2m_mean, t2m_stddev)]
minus_error = [x - y for x,y in zip(t2m_mean, t2m_stddev)]

In [ ]:
import pandas as pd
plt.figure(figsize=(12, 8))
plt.plot(x,t2m_mean, color='blue', linewidth=0.5)
plt.show()

## Export ERA5

In [77]:
# Export mean and stddev to file
sitename="am_0"
site_era5 = pd.DataFrame()
site_era5['dates'] = era5dates
site_era5['precip_mean'] = tp_mean
site_era5['precip_stddev'] = tp_stddev
site_era5['t2m_mean'] = t2m_mean
site_era5['t2_stddev'] = t2m_stddev

filename = "/Users/morgansteckler/Desktop/%s_era5.daily"%(sitename)
site_era5.to_csv(filename, index=False)

# ########################################
## GLDAS 2.1
# ########################################

In [ ]:
# GLDAS Net Longwave Radiation
gldas21_net_longwave_radiation = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
                          .select('Lwnet_tavg')
                          .filterBounds(fc)
                          .filter(ee.Filter.date('2017-01-01', '2020-12-31')))

# GLDAS Net Shortwave Radiation
gldas21_net_shortwave_radiation = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
                          .select('Swnet_tavg')
                          .filterBounds(fc)
                          .filter(ee.Filter.date('2017-01-01', '2020-12-31')))

#print(gldas21_net_longwave_radiation.size())
#print(gldas21_net_shortwave_radiation.size())


In [ ]:
## 3 hourly to daily time series -- take mean of values within the day
#1-day step
years = ee.List.sequence(2017, 2020)
period = ee.List.sequence(1, 30, 1)


def byYear(y):
    y = ee.Number(y)
    def dailymean(startDoy):
        startDoy = ee.Number(startDoy)
        return (gldas21_net_longwave_radiation
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(startDoy, startDoy, 'day_of_year'))
                .median())
    return period.map(dailymean)

mapped_doy = years.map(byYear).flatten()
gldas21_net_longwave_radiation = ee.ImageCollection.fromImages(mapped_doy)


def byYear(y):
    y = ee.Number(y)
    def dailymean(startDoy):
        startDoy = ee.Number(startDoy)
        return (gldas21_net_shortwave_radiation
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(startDoy, startDoy, 'day_of_year'))
                .median())
    return period.map(dailymean)

mapped_doy = years.map(byYear).flatten()
gldas21_net_shortwave_radiation = ee.ImageCollection.fromImages(mapped_doy)

In [ ]:
# net longwave radiation
# get area statistics
nlwr_mean_list = region_mean_ImageCollection(gldas21_net_longwave_radiation)
nlwr_median_list = region_median_ImageCollection(gldas21_net_longwave_radiation)
nlwr_stddev_list = region_stddev_ImageCollection(gldas21_net_longwave_radiation)
nlwr_percentile_list = region_percentiles_ImageCollection(gldas21_net_longwave_radiation)

# turn it into numpy array
nlwr_mean = newList(nlwr_mean_list, 'Lwnet_tavg')
nlwr_median = newList(nlwr_median_list, 'Lwnet_tavg')
nlwr_stddev = newList(nlwr_stddev_list, 'Lwnet_tavg')
nlwr_percentile = p_list(nlwr_percentile_list, 'Lwnet_tavg')



In [ ]:
# net shortwave radiation
# get area statistics
nswr_mean_list = region_mean_ImageCollection(gldas21_net_shortwave_radiation)
nswr_median_list = region_median_ImageCollection(gldas21_net_shortwave_radiation)
nswr_stddev_list = region_stddev_ImageCollection(gldas21_net_shortwave_radiation)
nswr_percentile_list = region_percentiles_ImageCollection(gldas21_net_shortwave_radiation)

# turn it into numpy array
nswr_mean = newList(nswr_mean_list, 'Swnet_tavg')
nswr_median = newList(nswr_median_list, 'Swnet_tavg')
nswr_stddev = newList(nswr_stddev_list, 'Swnet_tavg')
nswr_percentile = p_list(nswr_percentile_list, 'Swnet_tavg')

## Export GLDAS2.1

In [ ]:
# Export mean and stddev to file
site_gldas21 = pd.DataFrame()

site_gldas21['nlwrad_mean'] = nlwr_mean
site_gldas21['nlwrad_stddev'] = nlwr_stddev
site_gldas21['nswrad_mean'] = nswr_mean
site_gldas21['nswrad_stddev'] = nswr_stddev

filename = "/home/jbk/projects/climate/tropics/sentinel-2/data/gldas21/%s_gldas21.daily"%(sitename)
site_gldas21.to_csv(filename, index=False)

## Make few plots

In [ ]:
#Make lists with +/- SD
plus_error = [x + y for x,y in zip(tp_mean, tp_stddev)]
minus_error = [x - y for x,y in zip(tp_mean, tp_stddev)]

plt.axes([2, 0, 2, 1])
plt.grid(b = True, which = 'major', axis = 'x')
plt.fill_between(icdates, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
plt.plot(icdates, tp_mean, label='tp')

In [ ]:
def plot_mean_stddev(var_mean, var_stddev, var_name):
    #Make lists with +/- SD
    icdates=np.array(list(range(0,30)))
    plus_error = [x + y for x,y in zip(var_mean, var_stddev)]
    minus_error = [x - y for x,y in zip(var_mean, var_stddev)]

    plt.axes([2, 0, 2, 1])
    plt.grid(b = True, which = 'major', axis = 'x')
    plt.fill_between(icdates, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
    plt.plot(icdates, var_mean, label=var_name)
    plt.legend(loc = "upper right")

In [ ]:
plot_mean_stddev(tp_mean, tp_stddev, var_name="Total Precipitation")

In [ ]:
plot_mean_stddev(t2m_mean, t2m_stddev, var_name="2m Air Temperature")

In [ ]:
plot_mean_stddev(nlwr_mean, nlwr_stddev, var_name="Net Long Wave Radiation")

In [ ]:
plot_mean_stddev(nlwr_mean, nlwr_stddev, var_name="Net Short Wave Radiation")

In [ ]:
# Export mean and stddev to file
edata = np.column_stack((icdates, tp_mean, tp_stddev, t2m_mean, t2m_stddev))
site_era5 = pd.DataFrame()
site_era5['dates'] = icdates
site_era5['precip_mean'] = tp_mean
site_era5['precip_stddev'] = tp_stddev
site_era5['t2m_mean'] = t2m_mean
site_era5['t2_stddev'] = t2m_stddev
site_era5['nlwrad_mean'] = nlwr_mean
site_era5['nlwrad_stddev'] = nlwr_stddev
site_era5['nswrad_mean'] = nswr_mean
site_era5['nswrad_stddev'] = nswr_stddev

site_era5.to_csv('manaus_era5.daily', index=False)
#site_era5.to_csv('atto_era5.daily', index=False)
#site_era5.to_csv('corumba_era5.daily', index=False)
#site_era5.to_csv('mato_era5.daily', index=False)
#site_era5.to_csv('saop_era5.daily', index=False)

In [ ]:
# accumulate variables for Growing Degree Days calculation 
start_date = ee.Date('2016-01-01', 'UTC')
def maccum_byYear(y):
    def maccum_byMonth(m):
        return (ic
                .filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRage(m, m, 'month'))
                .
            )

In [ ]:
start_date = ee.Date('2016-01-01', 'UTC')
new_date = start_date.advance(2, 'month')
#print(start_date)
print(start_date, new_date)

In [ ]:
print(ee.Date)

In [ ]:
print(icdates)

In [ ]:
icdates

In [ ]:
gldasdates = ymdList(gldas21_net_longwave_radiation)

In [ ]:
print(gldasdates)